In [1]:
import re

regex_workflow = re.compile(r"(\w+)\{(.*)\}")
regex_condition = re.compile(r"((\w)([><])(\d+):)?(\w+)")

tryint = lambda i: int(i) if i.isdigit() else i
workflows, parts = {}, []
with open("Day19.txt") as file:
    for line in file.read().splitlines():
        if not line:
            continue
        if match := regex_workflow.match(line):
            name, conditions = match.groups()
            workflow = regex_condition.findall(conditions)
            workflows[name] = [tuple(map(tryint, flow[1:])) for flow in workflow]
        else:
            part = dict(zip("xmas", map(int, re.findall(r"\d+", line))))
            parts.append(part)

In [2]:
def run(x, m, a, s, workflow="in"):
    for variable, operator, value, result in workflows[workflow]:
        if not variable or eval(f"{variable}{operator}{value}"):
            if result in "AR":
                return result == "A"
            return run(x, m, a, s, workflow=result)

sum(sum(part.values()) for part in parts if run(**part))

346230

In [3]:
from math import prod

def solve(limits, workflow="in"):
    if workflow == "R":
        # if rejected, no combination is valid
        return 0
    elif workflow == "A":
        # if accepted, the count is the product of all intervals
        return prod((upper - lower) + 1 for lower, upper in limits.values())
    count = 0
    # loop on all workflow rules, fallback at the end
    for variable, operator, value, result in workflows[workflow]:
        if not variable:
            # fallback value with no condition attached
            count += solve(limits, result)
        else:
            # current interval for the targeted variable
            lower, upper = limits[variable]
            # split the interval between potential accepted/rejected possibilities according to the operator
            if operator == "<":
                alimits = (lower, value - 1)
                rlimits = (value, upper)
            elif operator == ">":
                alimits = (value + 1, upper)
                rlimits = (lower, value)
            # check if accepted interval is empty
            if sum(alimits) >= 0:
                limits = limits | {variable: alimits}
                count += solve(limits, result)
            # check if rejected interval is empty
            if sum(rlimits) >= 0:
                limits = limits | {variable: rlimits}
            else:
                # if rejected interval is empty, accepted interval covered all possibilities
                # no need to check further workflow rules
                break
    return count

solve(dict.fromkeys("xmas", (1, 4000)))

124693661917133